Test Momenta with mesh grids
===============

In [1]:
from __future__ import print_function
import numpy as np
import sftrident.qft as qft
print(np.__version__)
from sys import getsizeof
print(getsizeof(1e30*np.ones((2,1,1),dtype=np.complex)))

/Users/szabo/pyTableTest/venv/lib/python2.7/site-packages/scipy/special/__init__.py:640: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._ufuncs import *
/Users/szabo/pyTableTest/venv/lib/python2.7/site-packages/scipy/linalg/basic.py:17: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._solve_toeplitz import levinson
/Users/szabo/pyTableTest/venv/lib/python2.7/site-packages/scipy/linalg/__init__.py:207: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._decomp_update import *
/Users/szabo/pyTableTest/venv/lib/python2.7/site-packages/scipy/special/_ellip_harm.py:7: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._ellip_harm_2 import _ellipsoid, _ellipsoid_norm
/Users/szabo/pyTableTest/venv/lib/python2.7/site-packages/scipy/sparse/lil.py:19: RuntimeWarning: n

1.15.0
160


/Users/szabo/pyTableTest/venv/lib/python2.7/site-packages/scipy/ndimage/measurements.py:36: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from . import _ni_label


In [2]:
E = np.linspace(1.1,3.5,2)
C = np.linspace(-1.0,1.0,3)
Ph = np.linspace(0.0,2.0*np.pi,4)

E2 = np.linspace(1.1,3.5,5)
C2 = np.linspace(-1.0,1.0,6)
Ph2 = np.linspace(0.0,2.0*np.pi,7)


#print("E:",E)
print("E size: %1.2e"%getsizeof(E))

#print("C:",C)
print("C size: %1.2e"%getsizeof(C))
#print("Ph:",Ph)
print("Ph size: %1.2e"%getsizeof(Ph))



#Egrid,Cgrid,Phgrid,Egrid2,Cgrid2,Phgrid2 = np.meshgrid(E,C,Ph,E2,C2,Ph2)
Egrid,Cgrid,Phgrid = np.meshgrid(E,C,Ph,indexing='ij')


#print("Egrid:",Egrid)
print("Egrid size: %1.2e"%getsizeof(Egrid))
#print("Cgrid:",Cgrid)
print("Cgrid size: %1.2e"%getsizeof(Cgrid))
#print("Phgrid:",Phgrid)
print("Phgrid size: %1.2e"%getsizeof(Phgrid))


print("Egrid:",Egrid.shape)
print("Cgrid:",Egrid.shape)
print("Phgrid:",Egrid.shape)

E size: 1.12e+02
C size: 1.20e+02
Ph size: 1.28e+02
Egrid size: 3.20e+02
Cgrid size: 3.20e+02
Phgrid size: 3.20e+02
Egrid: (2, 3, 4)
Cgrid: (2, 3, 4)
Phgrid: (2, 3, 4)


In [3]:
def buildMomenta(e,c,p):
    r=np.sqrt(e**2 - 1.0)
    sC = np.sin(np.arccos(c))
    p0 = e
    px = r*sC*np.cos(p)
    py = r*sC*np.sin(p)
    pz = r*c
    return np.array([p0,px,py,pz])
    

momGrid = buildMomenta(Egrid,Cgrid,Phgrid)
#momGrid2 = buildMomenta(Egrid2,Cgrid2,Phgrid2)

print("momGrid",momGrid.shape)
print("momGrid size: %1.2e"%getsizeof(momGrid))   

momGrid (4, 2, 3, 4)
momGrid size: 9.12e+02


In [4]:
M = momGrid[0]**2 - momGrid[1]**2- momGrid[2]**2- momGrid[3]**2
print(np.isclose(M,np.ones(M.shape),rtol=1e-8).all()==True)

True


In [5]:
mkvGrid = qft.MinkowskiVector(momGrid)
Mmkv = mkvGrid*mkvGrid
test = np.isclose(np.asarray(Mmkv),np.ones(np.asarray(Mmkv).shape),rtol=1e-8)
print(test.all()==True)

True


In [6]:
for iE,elE in enumerate(E):
    for iC,elC in enumerate(C):
        for iPh,elPh in enumerate(Ph):
            tMom = buildMomenta(elE,elC,elPh)
            
            #print(elE,elC,elPh)
            
            EgridPKT = Egrid[iE,iC,iPh]
            CgridPKT = Cgrid[iE,iC,iPh]
            PhgridPKT = Phgrid[iE,iC,iPh]
            #print(type(tMom))
            #print(type(momGrid[:,iE,iC,iPh]))
            if np.isclose(tMom,momGrid[:,iE,iC,iPh]).all()==True:
                pass
            else:
                print("-"*20,"ERROR")
                print("tpara: ({},{},{})".format(elE,elC,elPh))
                print("Gpara: ({},{},{})".format(EgridPKT,CgridPKT,PhgridPKT))
                print("tmom: ({})".format(tMom))
                print("Gmom: ({})".format(momGrid[:,iE,iC,iPh]))
print("Done.")

Done.


In [7]:
Etest = (Egrid>1.5)
print("e",Etest)
Ctest = (Cgrid>=0)
print("c",Ctest)
test = Etest * Ctest
print("t",test)

testGrid = np.array([Egrid[test],Cgrid[test],Phgrid[test]])

print(Egrid[test].shape)
print(Cgrid[test].shape)
print(Phgrid[test].shape)


print(testGrid.shape)
ind = 0

for iE,elE in enumerate(E):
    for iC,elC in enumerate(C):
        for iPh,elPh in enumerate(Ph):
            if elE>1.5 and elC>=0.0:
                print("-"*20)
                print("single:",elE,elC,elPh)
                print("grid:  ",testGrid[:,ind])
                ind+=1
                
testMom = buildMomenta(*testGrid)
testMkv = qft.MinkowskiVector(testMom)
print(testMkv*testMkv)

e [[[False False False False]
  [False False False False]
  [False False False False]]

 [[ True  True  True  True]
  [ True  True  True  True]
  [ True  True  True  True]]]
c [[[False False False False]
  [ True  True  True  True]
  [ True  True  True  True]]

 [[False False False False]
  [ True  True  True  True]
  [ True  True  True  True]]]
t [[[False False False False]
  [False False False False]
  [False False False False]]

 [[False False False False]
  [ True  True  True  True]
  [ True  True  True  True]]]
(8,)
(8,)
(8,)
(3, 8)
--------------------
single: 3.5 0.0 0.0
grid:   [3.5 0.  0. ]
--------------------
single: 3.5 0.0 2.0943951023931953
grid:   [3.5       0.        2.0943951]
--------------------
single: 3.5 0.0 4.1887902047863905
grid:   [3.5       0.        4.1887902]
--------------------
single: 3.5 0.0 6.283185307179586
grid:   [3.5        0.         6.28318531]
--------------------
single: 3.5 1.0 0.0
grid:   [3.5 1.  0. ]
--------------------
single: 3.5 1.0 2.0